# Instalar

In [9]:
%pip install transformers sentence_transformers
%pip install sacremoses

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Librerías

In [10]:
# from google.colab import drive
import pandas as pd
import requests
from io import BytesIO
from sentence_transformers import SentenceTransformer, util
from transformers import MarianMTModel, MarianTokenizer

import ipywidgets as widgets
from IPython.display import display
import torch
import joblib
from joblib import load

from transformers import MarianMTModel, MarianTokenizer

import warnings
# Ignorar todas las advertencias
warnings.filterwarnings("ignore")

# Funciones

In [11]:
def get_mood(model, model_LR_Em, new_phrase):
  # Inicializar el modelo
  model = SentenceTransformer('intfloat/multilingual-e5-small')
  # Diccionario de etiquetas
  labels_dicc = {0: "Alegre", 1: "Neutral", 2: "Triste"}

  # Vectorización de la nueva frase usando el vectorizador con el que entrenamos el modelo
  new_phrase_vectorized = model.encode([new_phrase])

  # Usa el modelo entrenado para predecir la etiqueta de la frase
  new_prediction = model_LR_Em.predict(new_phrase_vectorized)[0]

  # Retorna la etiqueta predicha
  print(f"La frase '{new_phrase}' pertenece a la categoría: {labels_dicc[new_prediction]}")
  if labels_dicc[new_prediction] == "Alegre":
    return "My mood is happy and "
  elif labels_dicc[new_prediction] == "Neutral":
    return "My mood is neutral and "
  else:
    return "My mood is sad and "

In [12]:
def translate(model_es_en, tokenizer_es_en, model_en_es, tokenizer_en_es, text, target_language='en'):
# def translate(text, target_language='en'):
    if target_language == 'en':
        # Traducción de español a inglés
        tokenizer = tokenizer_es_en
        model = model_es_en
    else:
        # Traducción de inglés a español
        tokenizer = tokenizer_en_es
        model = model_en_es

    # Tokenización y traducción
    tokens = tokenizer(text, return_tensors="pt", padding=True)
    translated_tokens = model.generate(**tokens)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    return translated_text

In [13]:
# Función para obtener recomendaciones usando embeddings ya calculados
def get_recommendations(model, precomputed_embeddings, user_preference, titles, top_n=3):
    # Generar el embedding para la preferencia del usuario
    user_embedding = model.encode(user_preference, convert_to_tensor=True)

    # Calcular similitudes de coseno entre el embedding del usuario y las descripciones ya embebidas
    cosine_similarities = util.cos_sim(user_embedding, precomputed_embeddings)[0]
    similar_indices = cosine_similarities.argsort(descending=True)[:top_n].tolist()

    # Extraer las recomendaciones
    recommendations = [(titles.iloc[i], cosine_similarities[i].item()) for i in similar_indices]
    return recommendations

In [14]:
# Función para descargar y cargar el archivo
def download_and_load(url):
    response = requests.get(url)
    response.raise_for_status()  # Verifica que la descarga fue exitosa
    return torch.load(BytesIO(response.content), map_location='cpu')

In [15]:
def load_model_github(url):
    # Descarga el archivo desde GitHub
    response = requests.get(url)
    response.raise_for_status()  # Verifica que la descarga fue exitosa
    
    # Carga el contenido con joblib
    modelo = joblib.load(BytesIO(response.content))
    return modelo


# Programa Principal

In [ ]:
# URL del archivo en GitHub
url = "https://github.com/flaibani/NLP/raw/main/modelo_LR_Em.joblib"
# Cargar el modelo
model_LR_Em= load_model_github(url)

# URLs crudas de los archivos en GitHub
movie_url = 'https://github.com/flaibani/NLP/raw/main/movie_embeddings.pt'
game_url = 'https://github.com/flaibani/NLP/raw/main/game_embeddings.pt'
book_url = 'https://github.com/flaibani/NLP/raw/main/book_embeddings.pt'
# Cargar los embeddings
movie_embeddings = download_and_load(movie_url)
game_embeddings = download_and_load(game_url)
book_embeddings = download_and_load(book_url)
print("Modelos cargados con éxito.")

# Cargar los archivos .csv (Películas, Juegos y Libros)
df_movies = pd.read_csv('https://github.com/flaibani/NLP/raw/main/IMDB-Movie-Data.csv')
df_games = pd.read_csv('https://github.com/flaibani/NLP/raw/main/bgg_database.csv')
df_books = pd.read_csv('https://github.com/flaibani/NLP/raw/main/gutenberg_books_detailed.csv')

# Combinar columnas de interés y llenar los NaN con texto vacío
df_movies['full_description'] = (
    df_movies['Genre'].fillna('') + ' ' +
    df_movies['Description'].fillna('') + ' ' +
    df_movies['Director'].fillna('') + ' ' +
    df_movies['Actors'].fillna('')
)

df_games['full_description'] = (
    df_games['game_name'].fillna('') + ' ' +
    df_games['description'].fillna('') + ' ' +
    df_games['categories'].fillna('')
)

df_books['full_description'] = (
    df_books['Title'].fillna('') + ' ' +
    df_books['Author'].fillna('') + ' ' +
    df_books['Summary'].fillna('') + ' ' +
    df_books['Subjects'].fillna('')
)

movies_desc = df_movies['full_description']
games_desc = df_games['full_description']
books_desc = df_books['full_description']

# Inicializar el modelo de embeddings
model_mood = SentenceTransformer('intfloat/multilingual-e5-small')
model_pref = SentenceTransformer('intfloat/multilingual-e5-small')

# Cargar modelo y tokenizador para traducción
model_name_es_en = 'Helsinki-NLP/opus-mt-es-en'  # Modelo para español a inglés
model_es_en = MarianMTModel.from_pretrained(model_name_es_en)
tokenizer_es_en = MarianTokenizer.from_pretrained(model_name_es_en)

model_name_en_es = 'Helsinki-NLP/opus-mt-en-es'  # Modelo para inglés a español
model_en_es = MarianMTModel.from_pretrained(model_name_en_es)
tokenizer_en_es = MarianTokenizer.from_pretrained(model_name_en_es)

# Crear widgets de interfaz con tamaño ajustado
mood_input = widgets.Text(
    description="¿Cómo te sientes?",
    placeholder="Describe tu estado de ánimo (ej., feliz, pensativo...)",
    style={'description_width': '200px'},  # Ancho de la etiqueta
    layout=widgets.Layout(width='400px')  # Ancho de la caja de texto
)

preference_input = widgets.Text(
    description="¿Cuáles son tus preferencias?:",
    placeholder="Escribe tu preferencia (ej., aventura, comedia...)",
    style={'description_width': '200px'},
    layout=widgets.Layout(width='400px')
)

button = widgets.Button(description="Recomendaciones")
output = widgets.Output()
# Función para manejar la solicitud del usuario y mostrar resultados
def on_button_clicked(b):
    with output:
        output.clear_output()  # Limpiar salida anterior

        # Obtener estado de ánimo
        user_mood = mood_input.value
        user_mood_en = get_mood(model_mood, model_LR_Em, user_mood)
        # Obtener preferencia del usuario
        user_preference = preference_input.value
        user_preference_en = translate(model_es_en, tokenizer_es_en, model_en_es, tokenizer_en_es, user_preference, 'en')
        # Combinar preferencia y estado de ánimo
        user_preference_mood_en = user_preference_en + " " + user_mood_en

        # Obtener recomendaciones usando los embeddings ya calculados
        movie_recommendations = get_recommendations(model_pref, movie_embeddings, user_preference_mood_en, df_movies['Title'])
        game_recommendations = get_recommendations(model_pref,game_embeddings, user_preference_mood_en, df_games['game_name'])
        book_recommendations = get_recommendations(model_pref,book_embeddings, user_preference_mood_en, df_books['Title'])

        # Mostrar recomendaciones
        print("\nPelículas recomendadas:")
        for title, score in movie_recommendations:
            print(f"- {title} (Similitud: {score:.2f})")
            # Obtener la descripción y manejar valores nulos o vacíos
            description_text = df_movies[df_movies['Title'] == title]['Description'].iloc[0]
            if pd.notna(description_text) and description_text.strip() != "":
                description = translate(model_es_en, tokenizer_es_en, model_en_es, tokenizer_en_es, description_text, 'es')
            else:
                description = "Descripción no disponible."
            # print(f"  Descripción: {description}")
            # Imprimir la descripción en fragmentos
            print("  Descripción:")
            for i in range(0, len(description), 100):
                print(description[i:i+100])  # Imprimir fragmentos de 200 caracteres
            print("-" * 80)                  # Separador entre elementos

        print("\nJuegos de mesa recomendados:")
        for name, score in game_recommendations:
            print(f"- {name} (Similitud: {score:.2f})")

        print("\nLibros recomendados:")
        for title, score in book_recommendations:
            print(f"- {title} (Similitud: {score:.2f})")
            # Obtener el resumen y manejar valores nulos o vacíos
            description_text = df_books[df_books['Title'] == title]['Summary'].iloc[0]
            if pd.notna(description_text) and description_text.strip() != "":
                description = translate(model_es_en, tokenizer_es_en, model_en_es, tokenizer_en_es, description_text, 'es')
            else:
                description = "Descripción no disponible."
            # print(f"  Descripción: {description}")
            # Imprimir la descripción en fragmentos
            print("  Descripción:")
            for i in range(0, len(description), 100):
                print(description[i:i+100])  # Imprimir fragmentos de 200 caracteres
            print("-" * 80)                  # Separador entre elementos

# Eliminar el evento si fue asignado previamente
button.on_click(on_button_clicked)  # Remueve el vínculo previo si se vuelve a ejecutar la celda
button.on_click(on_button_clicked)  # Asigna un nuevo vínculo

# Mostrar interfaz
display(mood_input, preference_input, button, output)

Modelos cargados con éxito.


Text(value='', description='¿Cómo te sientes?', layout=Layout(width='400px'), placeholder='Describe tu estado …

Text(value='', description='¿Cuáles son tus preferencias?:', layout=Layout(width='400px'), placeholder='Escrib…

Button(description='Recomendaciones', style=ButtonStyle())

Output()